First, we import all the data from the data set, and label them as their own types:  COVID-19, Normal, LUNG Opacity, Viral Pneumonia. 

In [10]:
%pip install pillow numpy tqdm torch openpyxl timm



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 8.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.6/184.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 9.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.9/436.9 kB 9.0 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm

In [ ]:



CLASS_DIRS = {
    "COVID": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/COVID/images",
    "Normal": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/Normal/images",
    "Lung_Opacity": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/Lung_Opacity/images",
    "Viral_Pneumonia": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/Viral Pneumonia/images"
}

CLASS_LABELS = {
    "COVID": 0,
    "Normal": 1,
    "Lung_Opacity": 2,
    "Viral_Pneumonia": 3
}

def load_class_data(class_dir, label, target_size=(299, 299)):
    """加载单个类别的图像数据并添加标签"""
    images = []
    labels = []
    for filename in tqdm(os.listdir(class_dir), desc=f"Loading {label}"):
        if filename.endswith(".png"):
            image_path = os.path.join(class_dir, filename)
            img = Image.open(image_path).convert('L').resize(target_size)
            img_array = np.array(img) / 255.0  # 添加归一化
            images.append(img_array)
            labels.append(CLASS_LABELS[label])
    return np.array(images), np.array(labels)

def build_full_dataset():
    """构建完整数据集（带标签）"""
    all_images = []
    all_labels = []
    for class_name, class_dir in CLASS_DIRS.items():
        images, labels = load_class_data(class_dir, class_name)
        all_images.append(images)
        all_labels.append(labels)
    full_images = np.concatenate(all_images, axis=0)
    full_labels = np.concatenate(all_labels, axis=0)
    return full_images, full_labels


X, y = build_full_dataset()
print("Final dataset shape:", X.shape)  # (21165, 299, 299)
print("Labels shape:", y.shape)        # (21165,)

Loading Viral_Pneumonia: 100%|██████████| 1345/1345 [00:04<00:00, 293.03it/s]


After creating the dataframe, it's time for deep learning.
CNN:

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
# (num_samples, height, width)
X = X[:, np.newaxis, :, :]  # → (num_samples, 1, height, width)



X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.3, 
    stratify=y, 
    random_state=66
)

print("Train shape:", X_train.shape, y_train.shape)  # (16932, 1, 299, 299) (16932,)
print("Test shape:", X_test.shape, y_test.shape)     # (6349.5, 1, 299, 299) (4233,)


X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.LongTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.LongTensor(y_test)


train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

Train shape: (14815, 1, 299, 299) (14815,)
Test shape: (6350, 1, 299, 299) (6350,)


In [ ]:
batch_size = 32

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,        # 
    num_workers=8        # 
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False        # 
)


Now create the CNN using pytorch

In [ ]:

torch.set_num_threads(16)

In [ ]:

class COVID_CNN(nn.Module):
    def __init__(self, num_classes=4):
        super(COVID_CNN, self).__init__()
        
        # conv1
        self.conv1 = nn.Conv2d(
            in_channels=1,   # input
            out_channels=32, # output
            kernel_size=3,    # 3x3 kernel
            padding=1         # keep size
        )
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)  
        
        # conv2
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        
        # fullyconnected layer
        self.fc1 = nn.Linear(64 * 74 * 74, 512)  
        self.fc2 = nn.Linear(512, num_classes)
        
        # Dropout
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        # conv1 → ReLU → pool
        x = self.pool(F.relu(self.conv1(x)))  # (batch_size, 32, 149, 149)
        
        # conv2 → ReLU → pool
        x = self.pool(F.relu(self.conv2(x)))  # (batch_size, 64, 74, 74)
        
        # view → flatten
        x = x.view(-1, 64 * 74 * 74)
        
        # fc1 → ReLU -> Dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        
        # output layer
        x = self.fc2(x)
        return x

In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [9]:
model = COVID_CNN(num_classes=4).to(device)
print(model)

COVID_CNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=350464, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=4, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [ ]:



criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()  
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        
        
        train_loader_tqdm = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)
        
        for inputs, labels in train_loader_tqdm:
            inputs, labels = inputs.to(device), labels.to(device)
            
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            
            train_loader_tqdm.set_postfix(loss=running_loss/(total/len(labels)), acc=correct/total)
        
        
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}")




In [11]:
if os.environ.get('MallocStackLogging') == '1':
    os.environ['MallocStackLogging'] = '0'

In [12]:
train_model(model, train_loader, criterion, optimizer, epochs=10)

Epoch 1/10:   0%|          | 0/463 [00:00<?, ?it/s]Python(20465) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(20491) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(20492) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(20493) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(20494) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(20495) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(20496) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(20497) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(20498) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Epoch 1/10, Loss: 0.8862, Acc: 0.7134


Epoch 2/10:   0%|          | 0/463 [00:00<?, ?it/s]Python(22820) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(22821) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(22822) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(22823) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(22824) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(22825) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(22826) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(22827) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Epoch 2/10, Loss: 0.4644, Acc: 0.8200


Epoch 3/10:   0%|          | 0/463 [00:00<?, ?it/s]Python(24932) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(24933) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(24934) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(24935) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(24936) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(24937) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(24938) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(24939) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


...

Epoch 8/10, Loss: 0.1238, Acc: 0.9549


Epoch 9/10:   0%|          | 0/463 [00:00<?, ?it/s]Python(37886) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(37887) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(37888) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(37889) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(37890) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(37891) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(37892) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(37893) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Epoch 9/10, Loss: 0.0955, Acc: 0.9639


Epoch 10/10:   0%|          | 0/463 [00:00<?, ?it/s]Python(40085) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(40086) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(40087) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(40088) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(40089) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(40090) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(40091) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(40092) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
                                                                                      

Epoch 10/10, Loss: 0.0791, Acc: 0.9717


In [ ]:
def evaluate_model(model, test_loader):
    model.eval()  
    correct = 0
    total = 0
    with torch.no_grad():  
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    test_acc = correct / total
    print(f"Test Accuracy: {test_acc:.4f}")
    return test_acc

evaluate_model(model, test_loader)
from sklearn.metrics import classification_report

def get_classification_report(model, test_loader):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
    print(classification_report(y_true, y_pred, target_names=CLASS_NAMES))

CLASS_NAMES = ["COVID-19", "Normal", "Lung_Opacity", "Viral_Pneumonia"]
get_classification_report(model, test_loader)

Test Accuracy: 0.8668
                 precision    recall  f1-score   support

       COVID-19       0.92      0.89      0.91      1085
         Normal       0.85      0.92      0.88      3058
   Lung_Opacity       0.84      0.75      0.79      1804
Viral_Pneumonia       0.94      0.92      0.93       403

       accuracy                           0.87      6350
      macro avg       0.89      0.87      0.88      6350
   weighted avg       0.87      0.87      0.87      6350



In [13]:
torch.save(model.state_dict(), "covid_cnn_model.pth")
loaded_model = COVID_CNN(num_classes=4).to(device)
loaded_model.load_state_dict(torch.load("covid_cnn_model.pth"))

<All keys matched successfully>

Now try the classification method we learned.

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import gc

In [4]:
X_flat = X.reshape(X.shape[0], -1)
X = X_flat.astype(np.float32)

In [ ]:
del X_flat
gc.collect()


In [ ]:

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
del X
del y
gc.collect()

RF

In [7]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf_raw = rf.predict(X_test)
print("RF Accuracy:", accuracy_score(y_test, y_pred_rf_raw))

RF 未标准化准确率: 0.8577840774864163


In [ ]:
ada = AdaBoostClassifier(n_estimators=100, random_state=42)
ada.fit(X_train, y_train)
y_pred_ada_raw = ada.predict(X_test)
print("AdaBoost Accuracy:", accuracy_score(y_test, y_pred_ada_raw))

AdaBoost 未标准化准确率: 0.6810772501771793


Then it's pre-trained model

In [ ]:
import os

from tqdm import tqdm
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import timm


CLASS_DIRS = {
    "COVID": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/COVID/images",
    "Normal": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/Normal/images",
    "Lung_Opacity": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/Lung_Opacity/images",
    "Viral_Pneumonia": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/Viral Pneumonia/images"
}

CLASS_LABELS = {
    "COVID": 0,
    "Normal": 1,
    "Lung_Opacity": 2,
    "Viral_Pneumonia": 3
}


class COVIDDataset(Dataset):
    def __init__(self, class_dirs, class_labels, transform=None, target_size=(224, 224)):
        self.images = []
        self.labels = []
        self.transform = transform
        self.target_size = target_size
        for class_name, class_dir in class_dirs.items():
            for filename in os.listdir(class_dir):
                if filename.endswith(".png"):
                    image_path = os.path.join(class_dir, filename)
                    self.images.append(image_path)
                    self.labels.append(class_labels[class_name])

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image_path = self.images[idx]
        label = self.labels[idx]

        
        img = Image.open(image_path).convert('L')  
        img = img.convert('RGB')  
        img = img.resize(self.target_size)  

        
        if self.transform:
            img = self.transform(img)
        
        label = torch.tensor(label, dtype=torch.long)
        return img, label


IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])


full_dataset = COVIDDataset(CLASS_DIRS, CLASS_LABELS, transform=train_transform, target_size=(224, 224))


train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

num_train = int(train_ratio * len(full_dataset))
num_val = int(val_ratio * len(full_dataset))
num_test = len(full_dataset) - num_train - num_val

train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, [num_train, num_val, num_test], generator=torch.Generator().manual_seed(42)
)


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)


model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=len(CLASS_LABELS))



for param in model.parameters():
    param.requires_grad = False


for param in model.classifier.parameters():
    param.requires_grad = True


optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)


criterion = nn.CrossEntropyLoss()


def train_with_progress(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=10):
    model = model.to(device)
    best_val_acc = 0.0

    for epoch in range(num_epochs):
        
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        
        train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]", leave=False)
        for inputs, labels in train_pbar:
            inputs, labels = inputs.to(device), labels.to(device)

            
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            train_total += labels.size(0)
            train_correct += predicted.eq(labels).sum().item()

            
            train_pbar.set_postfix({
                "Loss": f"{loss.item():.4f}",
                "Acc": f"{100. * train_correct / train_total:.2f}%"
            })

        train_loss /= len(train_loader)
        train_acc = 100. * train_correct / train_total

        
        val_loss, val_acc = validate(model, val_loader, criterion, device)

        
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_efficientnet_B0.pth')
            print(f"Saved best model with Val Acc: {best_val_acc:.2f}%")


def validate(model, val_loader, criterion, device):
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        val_pbar = tqdm(val_loader, desc="Validating", leave=False)
        for inputs, labels in val_pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()

            val_pbar.set_postfix({
                "Loss": f"{loss.item():.4f}",
                "Acc": f"{100. * val_correct / val_total:.2f}%"
            })

    val_loss /= len(val_loader)
    val_acc = 100. * val_correct / val_total
    return val_loss, val_acc


device = torch.device("cpu")
model = model.to(device)
train_with_progress(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=5)

Epoch 1/5
  Train Loss: 2.5041, Train Acc: 37.54%
  Val Loss: 2.1244, Val Acc: 44.33%
Saved best model with Val Acc: 44.33%


Epoch 2/5
  Train Loss: 1.8261, Train Acc: 49.60%
  Val Loss: 1.6310, Val Acc: 53.91%
Saved best model with Val Acc: 53.91%


Epoch 3/5
  Train Loss: 1.4930, Train Acc: 56.25%
  Val Loss: 1.3825, Val Acc: 58.60%
Saved best model with Val Acc: 58.60%


Epoch 4/5
  Train Loss: 1.2962, Train Acc: 61.20%
  Val Loss: 1.2146, Val Acc: 63.83%
Saved best model with Val Acc: 63.83%


Epoch 5/5
  Train Loss: 1.1747, Train Acc: 64.10%
  Val Loss: 1.1510, Val Acc: 64.65%
Saved best model with Val Acc: 64.65%


In [ ]:
import os

from tqdm import tqdm
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import timm


CLASS_DIRS = {
    "COVID": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/COVID/images",
    "Normal": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/Normal/images",
    "Lung_Opacity": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/Lung_Opacity/images",
    "Viral_Pneumonia": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/Viral Pneumonia/images"
}

CLASS_LABELS = {
    "COVID": 0,
    "Normal": 1,
    "Lung_Opacity": 2,
    "Viral_Pneumonia": 3
}


class COVIDDataset(Dataset):
    def __init__(self, class_dirs, class_labels, transform=None, target_size=(224, 224)):
        self.images = []
        self.labels = []
        self.transform = transform
        self.target_size = target_size

        
        for class_name, class_dir in class_dirs.items():
            for filename in os.listdir(class_dir):
                if filename.endswith(".png"):
                    image_path = os.path.join(class_dir, filename)
                    self.images.append(image_path)
                    self.labels.append(class_labels[class_name])

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image_path = self.images[idx]
        label = self.labels[idx]

        
        img = Image.open(image_path).convert('L')  
        img = img.convert('RGB')  
        img = img.resize(self.target_size)  

        
        if self.transform:
            img = self.transform(img)
        
        label = torch.tensor(label, dtype=torch.long)
        return img, label


IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])


full_dataset = COVIDDataset(CLASS_DIRS, CLASS_LABELS, transform=train_transform, target_size=(224, 224))


train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

num_train = int(train_ratio * len(full_dataset))
num_val = int(val_ratio * len(full_dataset))
num_test = len(full_dataset) - num_train - num_val

train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, [num_train, num_val, num_test], generator=torch.Generator().manual_seed(42)
)


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)


model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=len(CLASS_LABELS))


for param in model.parameters():
    param.requires_grad = False


blocks_to_unfreeze = [
    model.classifier
]
for block in blocks_to_unfreeze:
    for param in block.parameters():
        param.requires_grad = True


optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)


criterion = nn.CrossEntropyLoss()


def train_with_progress(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=5):
    model = model.to(device)
    best_val_acc = 0.0
    patience = 2
    stale_epochs = 0

    for epoch in range(num_epochs):
        
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]", leave=False)
        for inputs, labels in train_pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            train_total += labels.size(0)
            train_correct += predicted.eq(labels).sum().item()

            train_pbar.set_postfix({
                "Loss": f"{loss.item():.4f}",
                "Acc": f"{100.*train_correct/train_total:.2f}%"
            })

        
        val_loss, val_acc = validate(model, val_loader, criterion, device)
        
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"  Train Loss: {train_loss/len(train_loader):.4f}, Acc: {100.*train_correct/train_total:.2f}%")
        print(f"  Val Loss: {val_loss:.4f}, Acc: {val_acc:.2f}%")

        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_efficientnet_B0_v2.pth')
            stale_epochs = 0
            print(f"Saved new best model with Acc: {val_acc:.2f}%")
        else:
            stale_epochs += 1
            if stale_epochs >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break


def validate(model, val_loader, criterion, device):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        val_pbar = tqdm(val_loader, desc="Validating", leave=False)
        for inputs, labels in val_pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            val_pbar.set_postfix({"Acc": f"{100.*correct/total:.2f}%"})

    return val_loss/len(val_loader), 100.*correct/total


device = torch.device("cpu")
model = model.to(device)
train_with_progress(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=5)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch 1/5
  Train Loss: 2.0847, Acc: 42.96%
  Val Loss: 1.7957, Acc: 48.30%
Saved new best model with Acc: 48.30%


Epoch 2/5
  Train Loss: 1.5776, Acc: 54.01%
  Val Loss: 1.4444, Acc: 57.50%
Saved new best model with Acc: 57.50%


Epoch 3/5
  Train Loss: 1.3719, Acc: 59.29%
  Val Loss: 1.3128, Acc: 61.12%
Saved new best model with Acc: 61.12%


Epoch 4/5
  Train Loss: 1.2174, Acc: 63.64%
  Val Loss: 1.1251, Acc: 65.56%
Saved new best model with Acc: 65.56%


Epoch 5/5
  Train Loss: 1.1117, Acc: 65.80%
  Val Loss: 1.0577, Acc: 68.71%
Saved new best model with Acc: 68.71%


In [2]:
import timm
print(timm.list_models('*efficientnetv2*'))

['efficientnetv2_l', 'efficientnetv2_m', 'efficientnetv2_rw_m', 'efficientnetv2_rw_s', 'efficientnetv2_rw_t', 'efficientnetv2_s', 'efficientnetv2_xl', 'gc_efficientnetv2_rw_t', 'tf_efficientnetv2_b0', 'tf_efficientnetv2_b1', 'tf_efficientnetv2_b2', 'tf_efficientnetv2_b3', 'tf_efficientnetv2_l', 'tf_efficientnetv2_m', 'tf_efficientnetv2_s', 'tf_efficientnetv2_xl']


In [ ]:
import os

from tqdm import tqdm
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import timm


CLASS_DIRS = {
    "COVID": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/COVID/images",
    "Normal": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/Normal/images",
    "Lung_Opacity": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/Lung_Opacity/images",
    "Viral_Pneumonia": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/Viral Pneumonia/images"
}

CLASS_LABELS = {
    "COVID": 0,
    "Normal": 1,
    "Lung_Opacity": 2,
    "Viral_Pneumonia": 3
}


class COVIDDataset(Dataset):
    def __init__(self, class_dirs, class_labels, transform=None, target_size=(224, 224)):
        self.images = []
        self.labels = []
        self.transform = transform
        self.target_size = target_size

        
        for class_name, class_dir in class_dirs.items():
            for filename in os.listdir(class_dir):
                if filename.endswith(".png"):
                    image_path = os.path.join(class_dir, filename)
                    self.images.append(image_path)
                    self.labels.append(class_labels[class_name])

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image_path = self.images[idx]
        label = self.labels[idx]

        
        img = Image.open(image_path).convert('L')  
        img = img.convert('RGB')  
        img = img.resize(self.target_size)  

        
        if self.transform:
            img = self.transform(img)
        
        label = torch.tensor(label, dtype=torch.long)
        return img, label


IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])


full_dataset = COVIDDataset(CLASS_DIRS, CLASS_LABELS, transform=train_transform, target_size=(224, 224))


train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

num_train = int(train_ratio * len(full_dataset))
num_val = int(val_ratio * len(full_dataset))
num_test = len(full_dataset) - num_train - num_val

train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, [num_train, num_val, num_test], generator=torch.Generator().manual_seed(42)
)


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)


model = timm.create_model('tf_efficientnetv2_b0', pretrained=True, num_classes=len(CLASS_LABELS))  # B0 版本


for param in model.parameters():
    param.requires_grad = False


blocks_to_unfreeze = [
    model.classifier         
]
for block in blocks_to_unfreeze:
    for param in block.parameters():
        param.requires_grad = True


optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)


criterion = nn.CrossEntropyLoss()


def train_with_progress(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=5):
    model = model.to(device)
    best_val_acc = 0.0
    patience = 2
    stale_epochs = 0

    for epoch in range(num_epochs):
        
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]", leave=False)
        for inputs, labels in train_pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            train_total += labels.size(0)
            train_correct += predicted.eq(labels).sum().item()

            train_pbar.set_postfix({
                "Loss": f"{loss.item():.4f}",
                "Acc": f"{100.*train_correct/train_total:.2f}%"
            })

        
        val_loss, val_acc = validate(model, val_loader, criterion, device)
        
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"  Train Loss: {train_loss/len(train_loader):.4f}, Acc: {100.*train_correct/train_total:.2f}%")
        print(f"  Val Loss: {val_loss:.4f}, Acc: {val_acc:.2f}%")

        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_efficientnet_v2_B0_v2.pth')
            stale_epochs = 0
            print(f"Saved new best model with Acc: {val_acc:.2f}%")
        else:
            stale_epochs += 1
            if stale_epochs >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break


def validate(model, val_loader, criterion, device):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        val_pbar = tqdm(val_loader, desc="Validating", leave=False)
        for inputs, labels in val_pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            val_pbar.set_postfix({"Acc": f"{100.*correct/total:.2f}%"})

    return val_loss/len(val_loader), 100.*correct/total


device = torch.device("cpu")
model = model.to(device)
train_with_progress(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=5)

Epoch 1/5
  Train Loss: 2.1187, Acc: 33.63%
  Val Loss: 1.7591, Acc: 41.97%
Saved new best model with Acc: 41.97%


Epoch 2/5
  Train Loss: 1.5342, Acc: 49.21%
  Val Loss: 1.4164, Acc: 52.27%
Saved new best model with Acc: 52.27%


Epoch 3/5
  Train Loss: 1.2756, Acc: 56.48%
  Val Loss: 1.2329, Acc: 58.19%
Saved new best model with Acc: 58.19%


Epoch 4/5
  Train Loss: 1.1387, Acc: 60.76%
  Val Loss: 1.0777, Acc: 61.85%
Saved new best model with Acc: 61.85%


Epoch 5/5
  Train Loss: 1.0272, Acc: 64.41%
  Val Loss: 0.9781, Acc: 63.93%
Saved new best model with Acc: 63.93%


In [ ]:

import os

from tqdm import tqdm
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import timm


CLASS_DIRS = {
    "COVID": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/COVID/images",
    "Normal": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/Normal/images",
    "Lung_Opacity": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/Lung_Opacity/images",
    "Viral_Pneumonia": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/Viral Pneumonia/images"
}

CLASS_LABELS = {
    "COVID": 0,
    "Normal": 1,
    "Lung_Opacity": 2,
    "Viral_Pneumonia": 3
}


class COVIDDataset(Dataset):
    def __init__(self, class_dirs, class_labels, transform=None, target_size=(224, 224)):
        self.images = []
        self.labels = []
        self.transform = transform
        self.target_size = target_size

        
        for class_name, class_dir in class_dirs.items():
            for filename in os.listdir(class_dir):
                if filename.endswith(".png"):
                    image_path = os.path.join(class_dir, filename)
                    self.images.append(image_path)
                    self.labels.append(class_labels[class_name])

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image_path = self.images[idx]
        label = self.labels[idx]

        
        img = Image.open(image_path).convert('L')  
        img = img.convert('RGB')  
        img = img.resize(self.target_size)  

        
        if self.transform:
            img = self.transform(img)
        
        label = torch.tensor(label, dtype=torch.long)
        return img, label


IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])


full_dataset = COVIDDataset(CLASS_DIRS, CLASS_LABELS, transform=train_transform, target_size=(224, 224))


train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

num_train = int(train_ratio * len(full_dataset))
num_val = int(val_ratio * len(full_dataset))
num_test = len(full_dataset) - num_train - num_val

train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, [num_train, num_val, num_test], generator=torch.Generator().manual_seed(42)
)


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)


model = timm.create_model('mobilevit_s', pretrained=True, num_classes=len(CLASS_LABELS))


for param in model.parameters():
    param.requires_grad = False


blocks_to_unfreeze = [
    model.head         
]
for block in blocks_to_unfreeze:
    for param in block.parameters():
        param.requires_grad = True


optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)


criterion = nn.CrossEntropyLoss()


def train_with_progress(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=5):
    model = model.to(device)
    best_val_acc = 0.0
    patience = 2
    stale_epochs = 0

    for epoch in range(num_epochs):
        
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]", leave=False)
        for inputs, labels in train_pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            train_total += labels.size(0)
            train_correct += predicted.eq(labels).sum().item()

            train_pbar.set_postfix({
                "Loss": f"{loss.item():.4f}",
                "Acc": f"{100.*train_correct/train_total:.2f}%"
            })

        
        val_loss, val_acc = validate(model, val_loader, criterion, device)
        
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"  Train Loss: {train_loss/len(train_loader):.4f}, Acc: {100.*train_correct/train_total:.2f}%")
        print(f"  Val Loss: {val_loss:.4f}, Acc: {val_acc:.2f}%")

        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_mobilevit_s.pth')
            stale_epochs = 0
            print(f"Saved new best model with Acc: {val_acc:.2f}%")
        else:
            stale_epochs += 1
            if stale_epochs >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break


def validate(model, val_loader, criterion, device):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        val_pbar = tqdm(val_loader, desc="Validating", leave=False)
        for inputs, labels in val_pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            val_pbar.set_postfix({"Acc": f"{100.*correct/total:.2f}%"})

    return val_loss/len(val_loader), 100.*correct/total


device = torch.device("cpu")
model = model.to(device)
train_with_progress(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=5)

Epoch 1/5
  Train Loss: 1.2117, Acc: 62.18%
  Val Loss: 1.0639, Acc: 66.73%
Saved new best model with Acc: 66.73%


Epoch 2/5
  Train Loss: 0.9993, Acc: 67.80%
  Val Loss: 0.9275, Acc: 69.53%
Saved new best model with Acc: 69.53%


Epoch 3/5
  Train Loss: 0.8913, Acc: 70.83%
  Val Loss: 0.8345, Acc: 72.87%
Saved new best model with Acc: 72.87%


Epoch 4/5
  Train Loss: 0.8265, Acc: 72.78%
  Val Loss: 0.7806, Acc: 74.39%
Saved new best model with Acc: 74.39%


Epoch 5/5
  Train Loss: 0.7841, Acc: 74.08%
  Val Loss: 0.7513, Acc: 75.11%
Saved new best model with Acc: 75.11%


In [ ]:
import timm
print(timm.list_models('*mobilevitv2*'))

['mobilevitv2_050', 'mobilevitv2_075', 'mobilevitv2_100', 'mobilevitv2_125', 'mobilevitv2_150', 'mobilevitv2_175', 'mobilevitv2_200']


In [ ]:

import os

from tqdm import tqdm
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import timm


CLASS_DIRS = {
    "COVID": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/COVID/images",
    "Normal": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/Normal/images",
    "Lung_Opacity": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/Lung_Opacity/images",
    "Viral_Pneumonia": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/Viral Pneumonia/images"
}

CLASS_LABELS = {
    "COVID": 0,
    "Normal": 1,
    "Lung_Opacity": 2,
    "Viral_Pneumonia": 3
}


class COVIDDataset(Dataset):
    def __init__(self, class_dirs, class_labels, transform=None, target_size=(224, 224)):
        self.images = []
        self.labels = []
        self.transform = transform
        self.target_size = target_size

        
        for class_name, class_dir in class_dirs.items():
            for filename in os.listdir(class_dir):
                if filename.endswith(".png"):
                    image_path = os.path.join(class_dir, filename)
                    self.images.append(image_path)
                    self.labels.append(class_labels[class_name])

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image_path = self.images[idx]
        label = self.labels[idx]

        
        img = Image.open(image_path).convert('L')  
        img = img.convert('RGB')  
        img = img.resize(self.target_size)  

        
        if self.transform:
            img = self.transform(img)
        
        label = torch.tensor(label, dtype=torch.long)
        return img, label


IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])


full_dataset = COVIDDataset(CLASS_DIRS, CLASS_LABELS, transform=train_transform, target_size=(224, 224))


train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

num_train = int(train_ratio * len(full_dataset))
num_val = int(val_ratio * len(full_dataset))
num_test = len(full_dataset) - num_train - num_val

train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, [num_train, num_val, num_test], generator=torch.Generator().manual_seed(42)
)


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)


model = timm.create_model('mobilevitv2_100', pretrained=True, num_classes=len(CLASS_LABELS))


for param in model.parameters():
    param.requires_grad = False


blocks_to_unfreeze = [
    model.head         
]
for block in blocks_to_unfreeze:
    for param in block.parameters():
        param.requires_grad = True


optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)


criterion = nn.CrossEntropyLoss()


def train_with_progress(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=5):
    model = model.to(device)
    best_val_acc = 0.0
    patience = 2
    stale_epochs = 0

    for epoch in range(num_epochs):
        
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]", leave=False)
        for inputs, labels in train_pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            train_total += labels.size(0)
            train_correct += predicted.eq(labels).sum().item()

            train_pbar.set_postfix({
                "Loss": f"{loss.item():.4f}",
                "Acc": f"{100.*train_correct/train_total:.2f}%"
            })

        
        val_loss, val_acc = validate(model, val_loader, criterion, device)
        
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"  Train Loss: {train_loss/len(train_loader):.4f}, Acc: {100.*train_correct/train_total:.2f}%")
        print(f"  Val Loss: {val_loss:.4f}, Acc: {val_acc:.2f}%")

        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_mobilevit_s_v2.pth')
            stale_epochs = 0
            print(f"Saved new best model with Acc: {val_acc:.2f}%")
        else:
            stale_epochs += 1
            if stale_epochs >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break


def validate(model, val_loader, criterion, device):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        val_pbar = tqdm(val_loader, desc="Validating", leave=False)
        for inputs, labels in val_pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            val_pbar.set_postfix({"Acc": f"{100.*correct/total:.2f}%"})

    return val_loss/len(val_loader), 100.*correct/total


device = torch.device("cpu")
model = model.to(device)
train_with_progress(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=5)

Epoch 1/5
  Train Loss: 1.2355, Acc: 64.01%
  Val Loss: 1.1110, Acc: 70.48%
Saved new best model with Acc: 70.48%


Epoch 2/5
  Train Loss: 1.0374, Acc: 70.95%
  Val Loss: 0.9509, Acc: 73.47%
Saved new best model with Acc: 73.47%


Epoch 3/5
  Train Loss: 0.9184, Acc: 72.92%
  Val Loss: 0.8663, Acc: 73.98%
Saved new best model with Acc: 73.98%


Epoch 4/5
  Train Loss: 0.8449, Acc: 74.29%
  Val Loss: 0.8012, Acc: 76.69%
Saved new best model with Acc: 76.69%


Epoch 5/5
  Train Loss: 0.7955, Acc: 75.15%
  Val Loss: 0.7464, Acc: 77.50%
Saved new best model with Acc: 77.50%


In [ ]:
import timm
print(timm.list_models('*edgenext*'))

['edgenext_base', 'edgenext_small', 'edgenext_small_rw', 'edgenext_x_small', 'edgenext_xx_small']


In [ ]:

import os

from tqdm import tqdm
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import timm


CLASS_DIRS = {
    "COVID": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/COVID/images",
    "Normal": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/Normal/images",
    "Lung_Opacity": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/Lung_Opacity/images",
    "Viral_Pneumonia": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/Viral Pneumonia/images"
}

CLASS_LABELS = {
    "COVID": 0,
    "Normal": 1,
    "Lung_Opacity": 2,
    "Viral_Pneumonia": 3
}


class COVIDDataset(Dataset):
    def __init__(self, class_dirs, class_labels, transform=None, target_size=(224, 224)):
        self.images = []
        self.labels = []
        self.transform = transform
        self.target_size = target_size

        
        for class_name, class_dir in class_dirs.items():
            for filename in os.listdir(class_dir):
                if filename.endswith(".png"):
                    image_path = os.path.join(class_dir, filename)
                    self.images.append(image_path)
                    self.labels.append(class_labels[class_name])

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image_path = self.images[idx]
        label = self.labels[idx]

        
        img = Image.open(image_path).convert('L')  
        img = img.convert('RGB')  
        img = img.resize(self.target_size)  

        
        if self.transform:
            img = self.transform(img)
        
        label = torch.tensor(label, dtype=torch.long)
        return img, label


IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])


full_dataset = COVIDDataset(CLASS_DIRS, CLASS_LABELS, transform=train_transform, target_size=(224, 224))


train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

num_train = int(train_ratio * len(full_dataset))
num_val = int(val_ratio * len(full_dataset))
num_test = len(full_dataset) - num_train - num_val

train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, [num_train, num_val, num_test], generator=torch.Generator().manual_seed(42)
)


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)


model = timm.create_model('edgenext_small', pretrained=True, num_classes=len(CLASS_LABELS))


for param in model.parameters():
    param.requires_grad = False


blocks_to_unfreeze = [
    model.head         
]
for block in blocks_to_unfreeze:
    for param in block.parameters():
        param.requires_grad = True


optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)


criterion = nn.CrossEntropyLoss()


def train_with_progress(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=5):
    model = model.to(device)
    best_val_acc = 0.0
    patience = 2
    stale_epochs = 0

    for epoch in range(num_epochs):
        
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]", leave=False)
        for inputs, labels in train_pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            train_total += labels.size(0)
            train_correct += predicted.eq(labels).sum().item()

            train_pbar.set_postfix({
                "Loss": f"{loss.item():.4f}",
                "Acc": f"{100.*train_correct/train_total:.2f}%"
            })

        
        val_loss, val_acc = validate(model, val_loader, criterion, device)
        
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"  Train Loss: {train_loss/len(train_loader):.4f}, Acc: {100.*train_correct/train_total:.2f}%")
        print(f"  Val Loss: {val_loss:.4f}, Acc: {val_acc:.2f}%")

        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'edgenext_small.pth')
            stale_epochs = 0
            print(f"Saved new best model with Acc: {val_acc:.2f}%")
        else:
            stale_epochs += 1
            if stale_epochs >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break


def validate(model, val_loader, criterion, device):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        val_pbar = tqdm(val_loader, desc="Validating", leave=False)
        for inputs, labels in val_pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            val_pbar.set_postfix({"Acc": f"{100.*correct/total:.2f}%"})

    return val_loss/len(val_loader), 100.*correct/total


device = torch.device("cpu")
model = model.to(device)
train_with_progress(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=5)

Epoch 1/5
  Train Loss: 1.0194, Acc: 58.42%
  Val Loss: 0.8288, Acc: 66.64%
Saved new best model with Acc: 66.64%


Epoch 2/5
  Train Loss: 0.7562, Acc: 70.36%
  Val Loss: 0.6878, Acc: 74.64%
Saved new best model with Acc: 74.64%


Epoch 3/5
  Train Loss: 0.6576, Acc: 75.19%
  Val Loss: 0.6215, Acc: 76.91%
Saved new best model with Acc: 76.91%


Epoch 4/5
  Train Loss: 0.6044, Acc: 77.86%
  Val Loss: 0.5778, Acc: 78.89%
Saved new best model with Acc: 78.89%


Epoch 5/5
  Train Loss: 0.5668, Acc: 79.37%
  Val Loss: 0.5415, Acc: 80.18%
Saved new best model with Acc: 80.18%


unfreeze last three layers

In [ ]:
import os

from tqdm import tqdm
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import timm


CLASS_DIRS = {
    "COVID": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/COVID/images",
    "Normal": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/Normal/images",
    "Lung_Opacity": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/Lung_Opacity/images",
    "Viral_Pneumonia": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/Viral Pneumonia/images"
}

CLASS_LABELS = {
    "COVID": 0,
    "Normal": 1,
    "Lung_Opacity": 2,
    "Viral_Pneumonia": 3
}


class COVIDDataset(Dataset):
    def __init__(self, class_dirs, class_labels, transform=None, target_size=(224, 224)):
        self.images = []
        self.labels = []
        self.transform = transform
        self.target_size = target_size

        
        for class_name, class_dir in class_dirs.items():
            for filename in os.listdir(class_dir):
                if filename.endswith(".png"):
                    image_path = os.path.join(class_dir, filename)
                    self.images.append(image_path)
                    self.labels.append(class_labels[class_name])

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image_path = self.images[idx]
        label = self.labels[idx]

        
        img = Image.open(image_path).convert('L')  
        img = img.convert('RGB')  
        img = img.resize(self.target_size)  

        
        if self.transform:
            img = self.transform(img)
        
        label = torch.tensor(label, dtype=torch.long)
        return img, label


IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])


full_dataset = COVIDDataset(CLASS_DIRS, CLASS_LABELS, transform=train_transform, target_size=(224, 224))


train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

num_train = int(train_ratio * len(full_dataset))
num_val = int(val_ratio * len(full_dataset))
num_test = len(full_dataset) - num_train - num_val

train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, [num_train, num_val, num_test], generator=torch.Generator().manual_seed(42)
)


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)


model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=len(CLASS_LABELS))



for param in model.parameters():
    param.requires_grad = False


layers = list(model.children())


for layer in layers[-3:]:
    for param in layer.parameters():
        param.requires_grad = True


optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)


criterion = nn.CrossEntropyLoss()


def train_with_progress(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=10):
    model = model.to(device)
    best_val_acc = 0.0

    for epoch in range(num_epochs):
        
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        
        train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]", leave=False)
        for inputs, labels in train_pbar:
            inputs, labels = inputs.to(device), labels.to(device)

            
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            train_total += labels.size(0)
            train_correct += predicted.eq(labels).sum().item()

            
            train_pbar.set_postfix({
                "Loss": f"{loss.item():.4f}",
                "Acc": f"{100. * train_correct / train_total:.2f}%"
            })

        train_loss /= len(train_loader)
        train_acc = 100. * train_correct / train_total

        
        val_loss, val_acc = validate(model, val_loader, criterion, device)

        
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'last_three_layers_best_efficientnet_B0.pth')
            print(f"Saved best model with Val Acc: {best_val_acc:.2f}%")


def validate(model, val_loader, criterion, device):
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        val_pbar = tqdm(val_loader, desc="Validating", leave=False)
        for inputs, labels in val_pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()

            val_pbar.set_postfix({
                "Loss": f"{loss.item():.4f}",
                "Acc": f"{100. * val_correct / val_total:.2f}%"
            })

    val_loss /= len(val_loader)
    val_acc = 100. * val_correct / val_total
    return val_loss, val_acc


device = torch.device("cpu")
model = model.to(device)
train_with_progress(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=5)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch 1/5
  Train Loss: 2.5078, Train Acc: 39.23%
  Val Loss: 1.9839, Val Acc: 47.13%
Saved best model with Val Acc: 47.13%


Epoch 2/5
  Train Loss: 1.7434, Train Acc: 52.15%
  Val Loss: 1.5075, Val Acc: 56.90%
Saved best model with Val Acc: 56.90%


Epoch 3/5
  Train Loss: 1.4336, Train Acc: 58.91%
  Val Loss: 1.2974, Val Acc: 62.67%
Saved best model with Val Acc: 62.67%


Epoch 4/5
  Train Loss: 1.2757, Train Acc: 62.94%
  Val Loss: 1.1733, Val Acc: 65.63%
Saved best model with Val Acc: 65.63%


Epoch 5/5
  Train Loss: 1.1560, Train Acc: 65.68%
  Val Loss: 1.0609, Val Acc: 67.30%
Saved best model with Val Acc: 67.30%


In [ ]:
import os

from tqdm import tqdm
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import timm


CLASS_DIRS = {
    "COVID": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/COVID/images",
    "Normal": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/Normal/images",
    "Lung_Opacity": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/Lung_Opacity/images",
    "Viral_Pneumonia": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/Viral Pneumonia/images"
}

CLASS_LABELS = {
    "COVID": 0,
    "Normal": 1,
    "Lung_Opacity": 2,
    "Viral_Pneumonia": 3
}


class COVIDDataset(Dataset):
    def __init__(self, class_dirs, class_labels, transform=None, target_size=(224, 224)):
        self.images = []
        self.labels = []
        self.transform = transform
        self.target_size = target_size

        
        for class_name, class_dir in class_dirs.items():
            for filename in os.listdir(class_dir):
                if filename.endswith(".png"):
                    image_path = os.path.join(class_dir, filename)
                    self.images.append(image_path)
                    self.labels.append(class_labels[class_name])

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image_path = self.images[idx]
        label = self.labels[idx]

        
        img = Image.open(image_path).convert('L')  
        img = img.convert('RGB')  
        img = img.resize(self.target_size)  

        
        if self.transform:
            img = self.transform(img)
        
        label = torch.tensor(label, dtype=torch.long)
        return img, label


IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])


full_dataset = COVIDDataset(CLASS_DIRS, CLASS_LABELS, transform=train_transform, target_size=(224, 224))


train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

num_train = int(train_ratio * len(full_dataset))
num_val = int(val_ratio * len(full_dataset))
num_test = len(full_dataset) - num_train - num_val

train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, [num_train, num_val, num_test], generator=torch.Generator().manual_seed(42)
)


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)


model = timm.create_model('tf_efficientnetv2_b0', pretrained=True, num_classes=len(CLASS_LABELS))



for param in model.parameters():
    param.requires_grad = False

 





layers = list(model.children())


for layer in layers[-3:]:
    for param in layer.parameters():
        param.requires_grad = True


optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)


criterion = nn.CrossEntropyLoss()


def train_with_progress(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=10):
    model = model.to(device)
    best_val_acc = 0.0

    for epoch in range(num_epochs):
        
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        
        train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]", leave=False)
        for inputs, labels in train_pbar:
            inputs, labels = inputs.to(device), labels.to(device)

            
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            train_total += labels.size(0)
            train_correct += predicted.eq(labels).sum().item()

            
            train_pbar.set_postfix({
                "Loss": f"{loss.item():.4f}",
                "Acc": f"{100. * train_correct / train_total:.2f}%"
            })

        train_loss /= len(train_loader)
        train_acc = 100. * train_correct / train_total

        
        val_loss, val_acc = validate(model, val_loader, criterion, device)

        
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'last_three_layers_best_efficientnet_V2_B0.pth')
            print(f"Saved best model with Val Acc: {best_val_acc:.2f}%")


def validate(model, val_loader, criterion, device):
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        val_pbar = tqdm(val_loader, desc="Validating", leave=False)
        for inputs, labels in val_pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()

            val_pbar.set_postfix({
                "Loss": f"{loss.item():.4f}",
                "Acc": f"{100. * val_correct / val_total:.2f}%"
            })

    val_loss /= len(val_loader)
    val_acc = 100. * val_correct / val_total
    return val_loss, val_acc


device = torch.device("cpu")
model = model.to(device)
train_with_progress(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=5)

Epoch 1/5
  Train Loss: 1.9864, Train Acc: 36.65%
  Val Loss: 1.5761, Val Acc: 46.60%
Saved best model with Val Acc: 46.60%


Epoch 2/5
  Train Loss: 1.4118, Train Acc: 51.97%
  Val Loss: 1.2558, Val Acc: 56.58%
Saved best model with Val Acc: 56.58%


Epoch 3/5
  Train Loss: 1.1995, Train Acc: 58.03%
  Val Loss: 1.1079, Val Acc: 60.24%
Saved best model with Val Acc: 60.24%


Epoch 4/5
  Train Loss: 1.0601, Train Acc: 62.28%
  Val Loss: 0.9936, Val Acc: 64.65%
Saved best model with Val Acc: 64.65%


Epoch 5/5
  Train Loss: 0.9809, Train Acc: 65.45%
  Val Loss: 0.9179, Val Acc: 66.35%
Saved best model with Val Acc: 66.35%


In [ ]:

import os

from tqdm import tqdm
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import timm


CLASS_DIRS = {
    "COVID": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/COVID/images",
    "Normal": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/Normal/images",
    "Lung_Opacity": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/Lung_Opacity/images",
    "Viral_Pneumonia": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/Viral Pneumonia/images"
}

CLASS_LABELS = {
    "COVID": 0,
    "Normal": 1,
    "Lung_Opacity": 2,
    "Viral_Pneumonia": 3
}


class COVIDDataset(Dataset):
    def __init__(self, class_dirs, class_labels, transform=None, target_size=(224, 224)):
        self.images = []
        self.labels = []
        self.transform = transform
        self.target_size = target_size

        
        for class_name, class_dir in class_dirs.items():
            for filename in os.listdir(class_dir):
                if filename.endswith(".png"):
                    image_path = os.path.join(class_dir, filename)
                    self.images.append(image_path)
                    self.labels.append(class_labels[class_name])

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image_path = self.images[idx]
        label = self.labels[idx]

        
        img = Image.open(image_path).convert('L')  
        img = img.convert('RGB')  
        img = img.resize(self.target_size)  

        
        if self.transform:
            img = self.transform(img)
        
        label = torch.tensor(label, dtype=torch.long)
        return img, label


IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])


full_dataset = COVIDDataset(CLASS_DIRS, CLASS_LABELS, transform=train_transform, target_size=(224, 224))


train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

num_train = int(train_ratio * len(full_dataset))
num_val = int(val_ratio * len(full_dataset))
num_test = len(full_dataset) - num_train - num_val

train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, [num_train, num_val, num_test], generator=torch.Generator().manual_seed(42)
)


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)


model = timm.create_model('mobilevit_s', pretrained=True, num_classes=len(CLASS_LABELS))


for param in model.parameters():
    param.requires_grad = False



layers = list(model.children())


for layer in layers[-3:]:
    for param in layer.parameters():
        param.requires_grad = True


optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)


criterion = nn.CrossEntropyLoss()


def train_with_progress(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=5):
    model = model.to(device)
    best_val_acc = 0.0
    patience = 2
    stale_epochs = 0

    for epoch in range(num_epochs):
        
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]", leave=False)
        for inputs, labels in train_pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            train_total += labels.size(0)
            train_correct += predicted.eq(labels).sum().item()

            train_pbar.set_postfix({
                "Loss": f"{loss.item():.4f}",
                "Acc": f"{100.*train_correct/train_total:.2f}%"
            })

        
        val_loss, val_acc = validate(model, val_loader, criterion, device)
        
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"  Train Loss: {train_loss/len(train_loader):.4f}, Acc: {100.*train_correct/train_total:.2f}%")
        print(f"  Val Loss: {val_loss:.4f}, Acc: {val_acc:.2f}%")

        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'last_three_layers_best_mobilevit_s.pth')
            stale_epochs = 0
            print(f"Saved new best model with Acc: {val_acc:.2f}%")
        else:
            stale_epochs += 1
            if stale_epochs >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break


def validate(model, val_loader, criterion, device):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        val_pbar = tqdm(val_loader, desc="Validating", leave=False)
        for inputs, labels in val_pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            val_pbar.set_postfix({"Acc": f"{100.*correct/total:.2f}%"})

    return val_loss/len(val_loader), 100.*correct/total


device = torch.device("cpu")
model = model.to(device)
train_with_progress(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=5)

Epoch 1/5
  Train Loss: 0.5039, Acc: 86.12%
  Val Loss: 0.2500, Acc: 92.12%
Saved new best model with Acc: 92.12%


Epoch 2/5
  Train Loss: 0.2356, Acc: 92.47%
  Val Loss: 0.1785, Acc: 94.11%
Saved new best model with Acc: 94.11%


Epoch 3/5
  Train Loss: 0.1838, Acc: 93.86%
  Val Loss: 0.1505, Acc: 95.15%
Saved new best model with Acc: 95.15%


Epoch 4/5
  Train Loss: 0.1554, Acc: 94.56%
  Val Loss: 0.1596, Acc: 94.83%


Epoch 5/5
  Train Loss: 0.1304, Acc: 95.52%
  Val Loss: 0.1528, Acc: 94.68%
Early stopping at epoch 5


In [ ]:

import os

from tqdm import tqdm
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import timm


CLASS_DIRS = {
    "COVID": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/COVID/images",
    "Normal": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/Normal/images",
    "Lung_Opacity": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/Lung_Opacity/images",
    "Viral_Pneumonia": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/Viral Pneumonia/images"
}

CLASS_LABELS = {
    "COVID": 0,
    "Normal": 1,
    "Lung_Opacity": 2,
    "Viral_Pneumonia": 3
}


class COVIDDataset(Dataset):
    def __init__(self, class_dirs, class_labels, transform=None, target_size=(224, 224)):
        self.images = []
        self.labels = []
        self.transform = transform
        self.target_size = target_size

        
        for class_name, class_dir in class_dirs.items():
            for filename in os.listdir(class_dir):
                if filename.endswith(".png"):
                    image_path = os.path.join(class_dir, filename)
                    self.images.append(image_path)
                    self.labels.append(class_labels[class_name])

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image_path = self.images[idx]
        label = self.labels[idx]

        
        img = Image.open(image_path).convert('L')  
        img = img.convert('RGB')  
        img = img.resize(self.target_size)  

        
        if self.transform:
            img = self.transform(img)
        
        label = torch.tensor(label, dtype=torch.long)
        return img, label


IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])


full_dataset = COVIDDataset(CLASS_DIRS, CLASS_LABELS, transform=train_transform, target_size=(224, 224))


train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

num_train = int(train_ratio * len(full_dataset))
num_val = int(val_ratio * len(full_dataset))
num_test = len(full_dataset) - num_train - num_val

train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, [num_train, num_val, num_test], generator=torch.Generator().manual_seed(42)
)


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)


model = timm.create_model('mobilevitv2_100', pretrained=True, num_classes=len(CLASS_LABELS))


for param in model.parameters():
    param.requires_grad = False



layers = list(model.children())


for layer in layers[-3:]:
    for param in layer.parameters():
        param.requires_grad = True


optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)


criterion = nn.CrossEntropyLoss()


def train_with_progress(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=5):
    model = model.to(device)
    best_val_acc = 0.0
    patience = 2
    stale_epochs = 0

    for epoch in range(num_epochs):
        
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]", leave=False)
        for inputs, labels in train_pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            train_total += labels.size(0)
            train_correct += predicted.eq(labels).sum().item()

            train_pbar.set_postfix({
                "Loss": f"{loss.item():.4f}",
                "Acc": f"{100.*train_correct/train_total:.2f}%"
            })

        
        val_loss, val_acc = validate(model, val_loader, criterion, device)
        
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"  Train Loss: {train_loss/len(train_loader):.4f}, Acc: {100.*train_correct/train_total:.2f}%")
        print(f"  Val Loss: {val_loss:.4f}, Acc: {val_acc:.2f}%")

        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'last_three_layers_best_mobilevit_s_v2.pth')
            stale_epochs = 0
            print(f"Saved new best model with Acc: {val_acc:.2f}%")
        else:
            stale_epochs += 1
            if stale_epochs >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break


def validate(model, val_loader, criterion, device):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        val_pbar = tqdm(val_loader, desc="Validating", leave=False)
        for inputs, labels in val_pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            val_pbar.set_postfix({"Acc": f"{100.*correct/total:.2f}%"})

    return val_loss/len(val_loader), 100.*correct/total


device = torch.device("cpu")
model = model.to(device)
train_with_progress(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=5)

Epoch 1/5
  Train Loss: 0.5060, Acc: 84.52%
  Val Loss: 0.2326, Acc: 93.07%
Saved new best model with Acc: 93.07%


Epoch 2/5
  Train Loss: 0.2257, Acc: 92.37%
  Val Loss: 0.1806, Acc: 93.86%
Saved new best model with Acc: 93.86%


Epoch 3/5
  Train Loss: 0.1795, Acc: 93.84%
  Val Loss: 0.1583, Acc: 94.71%
Saved new best model with Acc: 94.71%


Epoch 4/5
  Train Loss: 0.1532, Acc: 94.59%
  Val Loss: 0.1514, Acc: 94.83%
Saved new best model with Acc: 94.83%


Epoch 5/5
  Train Loss: 0.1287, Acc: 95.38%
  Val Loss: 0.1380, Acc: 95.21%
Saved new best model with Acc: 95.21%


In [ ]:

import os

from tqdm import tqdm
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import timm


CLASS_DIRS = {
    "COVID": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/COVID/images",
    "Normal": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/Normal/images",
    "Lung_Opacity": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/Lung_Opacity/images",
    "Viral_Pneumonia": "/Users/vanris/Documents/UG-DS6300/Project/Dataset/COVID-19_Radiography_Dataset/Viral Pneumonia/images"
}

CLASS_LABELS = {
    "COVID": 0,
    "Normal": 1,
    "Lung_Opacity": 2,
    "Viral_Pneumonia": 3
}


class COVIDDataset(Dataset):
    def __init__(self, class_dirs, class_labels, transform=None, target_size=(224, 224)):
        self.images = []
        self.labels = []
        self.transform = transform
        self.target_size = target_size

        
        for class_name, class_dir in class_dirs.items():
            for filename in os.listdir(class_dir):
                if filename.endswith(".png"):
                    image_path = os.path.join(class_dir, filename)
                    self.images.append(image_path)
                    self.labels.append(class_labels[class_name])

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image_path = self.images[idx]
        label = self.labels[idx]

        
        img = Image.open(image_path).convert('L')  
        img = img.convert('RGB')  
        img = img.resize(self.target_size)  

        
        if self.transform:
            img = self.transform(img)
        
        label = torch.tensor(label, dtype=torch.long)
        return img, label


IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])


full_dataset = COVIDDataset(CLASS_DIRS, CLASS_LABELS, transform=train_transform, target_size=(224, 224))


train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

num_train = int(train_ratio * len(full_dataset))
num_val = int(val_ratio * len(full_dataset))
num_test = len(full_dataset) - num_train - num_val

train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, [num_train, num_val, num_test], generator=torch.Generator().manual_seed(42)
)


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)


model = timm.create_model('edgenext_small', pretrained=True, num_classes=len(CLASS_LABELS))


for param in model.parameters():
    param.requires_grad = False



layers = list(model.children())


for layer in layers[-3:]:
    for param in layer.parameters():
        param.requires_grad = True



optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)


criterion = nn.CrossEntropyLoss()


def train_with_progress(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=5):
    model = model.to(device)
    best_val_acc = 0.0
    patience = 2
    stale_epochs = 0

    for epoch in range(num_epochs):
        
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]", leave=False)
        for inputs, labels in train_pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            train_total += labels.size(0)
            train_correct += predicted.eq(labels).sum().item()

            train_pbar.set_postfix({
                "Loss": f"{loss.item():.4f}",
                "Acc": f"{100.*train_correct/train_total:.2f}%"
            })

        
        val_loss, val_acc = validate(model, val_loader, criterion, device)
        
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"  Train Loss: {train_loss/len(train_loader):.4f}, Acc: {100.*train_correct/train_total:.2f}%")
        print(f"  Val Loss: {val_loss:.4f}, Acc: {val_acc:.2f}%")

        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'last_three_layers_edgenext_small.pth')
            stale_epochs = 0
            print(f"Saved new best model with Acc: {val_acc:.2f}%")
        else:
            stale_epochs += 1
            if stale_epochs >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break


def validate(model, val_loader, criterion, device):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        val_pbar = tqdm(val_loader, desc="Validating", leave=False)
        for inputs, labels in val_pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            val_pbar.set_postfix({"Acc": f"{100.*correct/total:.2f}%"})

    return val_loss/len(val_loader), 100.*correct/total


device = torch.device("cpu")
model = model.to(device)
train_with_progress(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=5)

Epoch 1/5
  Train Loss: 0.3569, Acc: 87.18%
  Val Loss: 0.2010, Acc: 93.10%
Saved new best model with Acc: 93.10%


Epoch 2/5
  Train Loss: 0.1851, Acc: 93.71%
  Val Loss: 0.1996, Acc: 92.88%


Epoch 3/5
  Train Loss: 0.1534, Acc: 94.47%
  Val Loss: 0.1656, Acc: 94.45%
Saved new best model with Acc: 94.45%


Epoch 4/5
  Train Loss: 0.1265, Acc: 95.63%
  Val Loss: 0.1466, Acc: 94.90%
Saved new best model with Acc: 94.90%


Epoch 5/5
  Train Loss: 0.1038, Acc: 96.30%
  Val Loss: 0.2052, Acc: 93.29%


In [ ]:
def test(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0.0
    test_correct = 0
    test_total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            test_total += labels.size(0)
            test_correct += predicted.eq(labels).sum().item()

    test_loss /= len(test_loader)
    test_acc = 100. * test_correct / test_total
    print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")



Loading models and do the testing

In [9]:
model_1 = timm.create_model('efficientnet_b0', pretrained=True, num_classes=len(CLASS_LABELS))
model_2 = timm.create_model('tf_efficientnetv2_b0', pretrained=True, num_classes=len(CLASS_LABELS))
model_3 = timm.create_model('mobilevit_s', pretrained=True, num_classes=len(CLASS_LABELS))
model_4 = timm.create_model('mobilevitv2_100', pretrained=True, num_classes=len(CLASS_LABELS))
model_5 = timm.create_model('edgenext_small', pretrained=True, num_classes=len(CLASS_LABELS))

model_13 = timm.create_model('efficientnet_b0', pretrained=True, num_classes=len(CLASS_LABELS))
model_23 = timm.create_model('tf_efficientnetv2_b0', pretrained=True, num_classes=len(CLASS_LABELS))
model_33 = timm.create_model('mobilevit_s', pretrained=True, num_classes=len(CLASS_LABELS))
model_43 = timm.create_model('mobilevitv2_100', pretrained=True, num_classes=len(CLASS_LABELS))
model_53 = timm.create_model('edgenext_small', pretrained=True, num_classes=len(CLASS_LABELS))



In [ ]:

model_1.load_state_dict(torch.load('/Users/vanris/Documents/UG-DS6300/Project/best_efficientnet_B0_v2.pth'))
model_2.load_state_dict(torch.load('/Users/vanris/Documents/UG-DS6300/Project/best_efficientnet_v2_B0_v2.pth'))
model_3.load_state_dict(torch.load('/Users/vanris/Documents/UG-DS6300/Project/best_mobilevit_s.pth'))
model_4.load_state_dict(torch.load('/Users/vanris/Documents/UG-DS6300/Project/best_mobilevit_s_v2.pth'))
model_5.load_state_dict(torch.load('/Users/vanris/Documents/UG-DS6300/Project/edgenext_small.pth'))

model_13.load_state_dict(torch.load('/Users/vanris/Documents/UG-DS6300/Project/last_three_layers_best_efficientnet_B0.pth'))
model_23.load_state_dict(torch.load('/Users/vanris/Documents/UG-DS6300/Project/last_three_layers_best_efficientnet_V2_B0.pth'))
model_33.load_state_dict(torch.load('/Users/vanris/Documents/UG-DS6300/Project/last_three_layers_best_mobilevit_s.pth'))
model_43.load_state_dict(torch.load('/Users/vanris/Documents/UG-DS6300/Project/last_three_layers_best_mobilevit_s_v2.pth'))
model_53.load_state_dict(torch.load('/Users/vanris/Documents/UG-DS6300/Project/last_three_layers_edgenext_small.pth'))



<All keys matched successfully>

In [ ]:

test(model_1, test_loader, criterion, device)

Test Loss: 1.0572, Test Acc: 67.41%


In [15]:
test(model_2, test_loader, criterion, device)

Test Loss: 1.0041, Test Acc: 63.00%


In [16]:
test(model_3, test_loader, criterion, device)

Test Loss: 0.7707, Test Acc: 73.58%


In [17]:
test(model_4, test_loader, criterion, device)

Test Loss: 0.7543, Test Acc: 77.36%


In [18]:
test(model_5, test_loader, criterion, device)

Test Loss: 0.5454, Test Acc: 80.35%


In [19]:
test(model_13, test_loader, criterion, device)

Test Loss: 1.1475, Test Acc: 64.48%


In [20]:
test(model_23, test_loader, criterion, device)

Test Loss: 0.9140, Test Acc: 67.73%


In [21]:
test(model_33, test_loader, criterion, device)

Test Loss: 0.1574, Test Acc: 95.18%


In [22]:
test(model_43, test_loader, criterion, device)

Test Loss: 0.1403, Test Acc: 95.28%


In [23]:
test(model_53, test_loader, criterion, device)

Test Loss: 0.1385, Test Acc: 95.28%
